
# Big‑Idea Round‑Out Notebook v3  

**Run → all** to produce:  

* High‑precision identity tests (exact & approximate)  
* Prime‑sieve heat‑map mod 137 with optional shuffle  
* χ² statistic on the 135 coprime residue classes (skips 0)  
* CSV / PNG outputs and a ZIP reproducibility bundle  

Toggle behaviour in the “SETTINGS” cell below; no other edits needed.


In [12]:

# ═════════════ SETTINGS ═════════════
limit        = 1_500_000     # sieve upper bound (≈ 114k primes)
grid_mode    = 'crop'        # 'crop' | 'autosize' | 'fixed'
shuffle_rows = True         # True → random order (kills stripes)
sample_size  = None          # int or None; None = use all primes
tol_exact    = '1e-50'       # tolerance for exact identities
tol_loose    = '1e-6'        # tolerance for near‑miss identities
fixed_size   = 200           # used only if grid_mode == 'fixed'
# ═══════════════════════════════════


In [13]:

from decimal import Decimal, getcontext
import numpy as np, matplotlib.pyplot as plt
getcontext().prec = 70

pi  = Decimal('3.14159265358979323846264338327950288419716939937510')
phi = Decimal('1.61803398874989484820458683436563811772030917980576')
sqrt2 = Decimal(2).sqrt()
sqrt5 = Decimal(5).sqrt()
alpha_inv = Decimal('137.035999084')  # CODATA 2018
print('Constants ready.\n')


Constants ready.



In [14]:

def delta(a, b): return abs(a - b)

def check(label, lhs, rhs, tol):
    d = delta(lhs, rhs)
    status = 'PASS' if d <= Decimal(tol) else 'FAIL'
    print(f'{status:<4}  {label:<24}  Δ = {d}')
    return [label, str(lhs), str(rhs), str(d), tol, status]


In [15]:
# ▼ exact equalities  (should PASS at tol_exact)
identities_exact = [
    ('φ²  vs  φ+1',          phi**2,         phi + 1),
    ('√2² vs 2',             sqrt2**2,       Decimal(2)),
    ('Binet check',          (1+sqrt5)/2,    phi),  # exact by def
]

# ▼ near‑miss conjectures (should FAIL at tol_exact, PASS at tol_loose)
identities_loose = [
    ('π² vs 2πφ',            pi**2,          2*pi*phi),
    ('α⁻¹≈π·φ²',             alpha_inv,      pi*phi**2),
]
rows = [['label','lhs','rhs','delta','rel_err','tolerance','status']]

print('── Exact identities ──')
for lab,lhs,rhs in identities_exact:
    d  = delta(lhs, rhs)
    re = d / rhs            # ←← add this line
    status = 'PASS' if d <= Decimal(tol_exact) else 'FAIL'
    print(f'{status:<4}  {lab:<24}  Δ={d}  rel_err={re}')
    rows.append([lab, str(lhs), str(rhs), str(d), str(re), tol_exact, status])

print('\n── Near‑miss identities ──')
for lab,lhs,rhs in identities_loose:
    rows.append(check(lab,lhs,rhs,tol_loose))

import csv
with open('identity_results.csv','w',newline='') as f:
    csv.writer(f).writerows(rows)
print('\nidentity_results.csv saved.\n')

── Exact identities ──
PASS  φ²  vs  φ+1               Δ=6.399929423932225795E-51  rel_err=2.444555514341575541319954214150999785918415280529984693350532767995313E-51
PASS  √2² vs 2                  Δ=1E-69  rel_err=5E-70
PASS  Binet check               Δ=2.862135448622705260E-51  rel_err=1.768896987654760266972267481255895227753525472202818506487691727076999E-51

── Near‑miss identities ──
FAIL  π² vs 2πφ                 Δ = 0.296802983541161012784527026608246548349979179201517368417547597360025
FAIL  α⁻¹≈π·φ²                  Δ = 128.8112027380949469457278476034782982739709913074037843956879358932960

identity_results.csv saved.



In [18]:
import random
import numpy as np

# sieve primes < limit
sieve = np.ones(limit, dtype=bool)
primes = []
for p in range(2, limit):
    if sieve[p]:
        primes.append(p)
        sieve[p*p::p] = False
print(f'{len(primes):,} primes found below {limit}.')

# optional random shuffle (destroys stripe direction)
if shuffle_rows:
    primes = random.sample(primes, len(primes))

# optional random sample size
if sample_size:
    primes = random.sample(primes, sample_size)
    print(f'Random sample of size {sample_size} selected.')

114,155 primes found below 1500000.


In [19]:

modulus = 137

# decide matrix size
if grid_mode == 'autosize':
    size = math.ceil(math.sqrt(len(primes)))
elif grid_mode == 'fixed':
    size = fixed_size
else:  # crop first, fill then crop later
    size = fixed_size

mat = np.zeros((size,size), dtype=int)
for i,p in enumerate(primes[: size*size ]):
    mat[i//size, i%size] = p % modulus

# crop empty margins if requested
if grid_mode == 'crop':
    nzr = np.where(mat.sum(1)!=0)[0]; nzc = np.where(mat.sum(0)!=0)[0]
    mat = mat[nzr[:,None], nzc]

plt.figure(figsize=(6,6))
plt.imshow(mat, interpolation='nearest')
plt.axis('off')
plt.title(f'Prime residues mod {modulus}  • {len(primes):,} primes  • {grid_mode}')
plt.savefig('prime_heatmap.png', dpi=300, bbox_inches='tight')
plt.close()
print('prime_heatmap.png saved.\n')


prime_heatmap.png saved.



In [20]:

from collections import Counter
coprime_residues = [p % modulus for p in primes if p % modulus != 0]
expected = len(coprime_residues) / (modulus - 1)  # 135 classes
chi2 = sum((c - expected)**2 / expected for c in Counter(coprime_residues).values())
print(f'χ² = {chi2:.2f}  for 134 dof  (residue 0 excluded)\n')


χ² = 24.12  for 134 dof  (residue 0 excluded)



In [21]:
# ── Residue‑class χ² spectrogram ─────────────────────────────────────
import matplotlib.pyplot as plt
from collections import Counter
import numpy as np

window_size = 10_000      # how many primes per slice
step        = 5_000       # slide half‑overlapping windows
modulus     = 137

# list of coprime residues for all primes (skip 0 class)
res_seq = [p % modulus for p in primes if p % modulus != 0]

chi_vals = []
starts   = []

for start in range(0, len(res_seq) - window_size + 1, step):
    slice_ = res_seq[start : start + window_size]
    counts = Counter(slice_)
    exp    = window_size / (modulus - 1)      # 135 classes
    chi2   = sum((c - exp)**2 / exp for c in counts.values())
    chi_vals.append(chi2)
    starts.append(start + window_size//2)      # mid‑point for x‑axis

plt.figure(figsize=(8,4))
plt.plot(starts, chi_vals, lw=1)
plt.axhline(modulus-2, color='k', ls='--', alpha=0.6, label='d.o.f.=134')
plt.xlabel('prime index')
plt.ylabel('χ² (window)')
plt.title(f'Sliding‑window χ² for primes mod {modulus}  (window {window_size}, step {step})')
plt.legend()
plt.tight_layout()
plt.savefig('chi2_spectrogram.png', dpi=300)
plt.close()
print('chi2_spectrogram.png saved.')


chi2_spectrogram.png saved.


In [22]:
with open('README.txt','w') as f:
    f.write('Big‑Idea Bundle v3\nGenerated: ' + datetime.utcnow().isoformat() + 'Z\n\n')
    f.write('FILES:\n- identity_results.csv\n- prime_heatmap.png\n')

with zipfile.ZipFile('big_idea_bundle_v3.zip','w') as z:
    for fname in ['identity_results.csv','prime_heatmap.png','README.txt']:
        z.write(fname)
print('big_idea_bundle_v3.zip created.')

big_idea_bundle_v3.zip created.
